In [43]:
import pandas as pd
import numpy as np
import plotly.express as px

final_cost_report = pd.read_csv("final_cost_report3.csv")


In [54]:
# Reemplazar strings "nan" por NaN reales
final_cost_report.replace("nan", np.nan, inplace=True)

# Filtrar solo cuentas generales
cuentas_generales = final_cost_report[final_cost_report["REF"] == "Cuenta General"].copy()

# Agrupar por TIPO incluyendo COMMITMENTS
resumen_tipo = cuentas_generales.groupby("TIPO", as_index=False)[["COST TO DATE", "COMMITMENTS", "BUDGET"]].sum()

# Calcular TOTAL CTD como COST TO DATE + COMMITMENTS
resumen_tipo["TOTAL CTD"] = resumen_tipo["COST TO DATE"] + resumen_tipo["COMMITMENTS"]

# Calcular EFC (puede ser igual al TOTAL CTD si no hay otro método)
resumen_tipo["EFC"] = resumen_tipo["TOTAL CTD"]

# Calcular VARIANCE correctamente
resumen_tipo["VARIANCE"] = resumen_tipo["BUDGET"] - resumen_tipo["TOTAL CTD"]

# Calcular INDEX (% ejecución de presupuesto gastado hasta la fecha SIN commitments)
resumen_tipo["INDEX"] = (resumen_tipo["COST TO DATE"] / resumen_tipo["BUDGET"] * 100).round().astype(int)

# Reordenar columnas para mostrar todo
resumen_tipo = resumen_tipo[["TIPO", "COST TO DATE", "COMMITMENTS", "TOTAL CTD", "EFC", "BUDGET", "VARIANCE", "INDEX"]]

# Calcular fila total
fila_total = resumen_tipo[["COST TO DATE", "COMMITMENTS", "TOTAL CTD", "EFC", "BUDGET", "VARIANCE"]].sum()
fila_total["TIPO"] = "TOTAL"
fila_total["INDEX"] = ""  # Dejar en blanco para la fila total
resumen_tipo = pd.concat([resumen_tipo, pd.DataFrame([fila_total])], ignore_index=True)

# Formatear con separadores de miles
def formatear(val):
    try:
        return f"${float(val):,.0f}"
    except:
        return val

for col in ["COST TO DATE", "COMMITMENTS", "TOTAL CTD", "EFC", "BUDGET", "VARIANCE"]:
    resumen_tipo[col] = resumen_tipo[col].apply(formatear)

# Mostrar tabla en consola
print("\n" + "-"*130)
print("{:<18} {:>15} {:>15} {:>15} {:>15} {:>15} {:>15} {:>8}".format(
    "Concept", "COST TO DATE", "COMMITMENTS", "TOTAL CTD", "EFC", "BUDGET", "DIFFERENCE $", "INDEX"
))
print("-"*130)
for _, row in resumen_tipo.iterrows():
    print("{:<18} {:>15} {:>15} {:>15} {:>15} {:>15} {:>15} {:>8}".format(
        row["TIPO"], row["COST TO DATE"], row["COMMITMENTS"],
        row["TOTAL CTD"], row["EFC"], row["BUDGET"], row["VARIANCE"], row["INDEX"]
    ))
print("-"*130)

def formatear(val):
    return f"${val:,.0f}"

def imprimir_tabla(df, titulo):
    # Definir los anchos de columna
    widths = [8, 15, 15, 15]
    headers = ["ACCT", "COST TO DATE", "BUDGET", "DIFFERENCE"]

    # Borde superior
    print("\n" + "═"*60)
    print(f"{titulo}".center(60))
    print("═"*60)
    # Encabezados
    header_row = "║ {:^8} ║ {:^15} ║ {:^15} ║ {:^15} ║".format(*headers)
    print(header_row)
    print("╠" + "════════╬" + "═══════════════╬"*2 + "═══════════════╣")
    # Filas
    for _, row in df.iterrows():
        print("║ {:>8} ║ {:>15} ║ {:>15} ║ {:>15} ║".format(
            row['ACCT'], row['COST TO DATE'], row['BUDGET'], row['VARIANCE']
        ))
    # Borde inferior
    print("═"*60)

# Beneficiadas (Top 15)
beneficiadas = cuentas_generales.sort_values("VARIANCE", ascending=False).head(15).copy()
beneficiadas["COST TO DATE"] = beneficiadas["COST TO DATE"].apply(formatear)
beneficiadas["BUDGET"] = beneficiadas["BUDGET"].apply(formatear)
beneficiadas["VARIANCE"] = beneficiadas["VARIANCE"].apply(formatear)

imprimir_tabla(beneficiadas, "Cuentas Más Beneficiadas")

# Impactadas (Bottom 15)
impactadas = cuentas_generales.sort_values("VARIANCE", ascending=True).head(15).copy()
impactadas["COST TO DATE"] = impactadas["COST TO DATE"].apply(formatear)
impactadas["BUDGET"] = impactadas["BUDGET"].apply(formatear)
impactadas["VARIANCE"] = impactadas["VARIANCE"].apply(formatear)

imprimir_tabla(impactadas, "Cuentas Más Impactadas")






----------------------------------------------------------------------------------------------------------------------------------
Concept               COST TO DATE     COMMITMENTS       TOTAL CTD             EFC          BUDGET    DIFFERENCE $    INDEX
----------------------------------------------------------------------------------------------------------------------------------
Above the Line         $37,179,095              $0     $37,179,095     $37,179,095     $35,701,887     $-1,477,207      104
Below the Line         $68,434,405              $0     $68,434,405     $68,434,405     $69,999,853      $1,565,448       98
Others                 $14,267,410      $1,073,642     $15,341,052     $15,341,052     $14,989,184       $-351,868       95
Post                   $14,491,072              $0     $14,491,072     $14,491,072     $14,406,986        $-84,086      101
TOTAL                 $134,371,983      $1,073,642    $135,445,624    $135,445,624    $135,097,911       $-347,713   

In [50]:

# Filtrar solo cuentas generales si quieres
df = final_cost_report[final_cost_report["REF"] == "Cuenta General"].copy()

# Agrupar por TIPO y sumar COST TO DATE
resumen = df.groupby("TIPO", as_index=False)["COST TO DATE"].sum()
resumen = resumen.sort_values("COST TO DATE", ascending=False)

# Gráfico de rosca 
fig = px.pie(
    resumen,
    values="COST TO DATE",
    names="TIPO",
    title="<b>Porcentaje de uso por TIPO </b>",
    hole=0.5,  # Hace la rosca
)

fig.update_traces(
    textposition='inside',
    textinfo='percent+label',
    pull=[0.02]*len(resumen),  
    marker=dict(line=dict(color='white', width=2))
)

fig.update_layout(
    showlegend=True,
    title_font_size=22,
    legend_title_text='Tipo de Cuenta',
    legend=dict(font=dict(size=14))
)

fig.show()

La gráfica muestra la distribución porcentual del uso del presupuesto real (Cost to Date) por tipo de cuenta general. Cada segmento representa un tipo de gasto dentro del proyecto, permitiendo visualizar de forma clara cuáles categorías concentran la mayor parte del presupuesto ejecutado hasta la fecha. Esta visualización facilita identificar rápidamente los rubros que absorben más recursos y analizar si la distribución es acorde a los objetivos y prioridades del proyecto. El gráfico es útil para detectar posibles desviaciones, oportunidades de optimización y áreas donde se puede implementar una gestión financiera más eficiente

## Hipotesis

### Las etapas con mayor presupuesto son tambien los que generan mayores ahorros.

In [28]:
final_cost_report

,ACCT,TIPO,BUDGET,COST TO DATE,REF,Etapa,VARIANCE
0,1100,Above the Line,10065000.00,1.006500e+07,Cuenta General,NaN,-0.080
1,1200,Above the Line,9449620.00,9.361342e+06,Cuenta General,NaN,88277.690
2,1300,Above the Line,2810543.88,2.663851e+06,Cuenta General,NaN,146693.350
3,1400,Above the Line,11960556.35,1.389992e+07,Cuenta General,NaN,-1939365.675
4,1600,Above the Line,1416167.22,1.188980e+06,Cuenta General,NaN,227187.455
...,...,...,...,...,...,...,...
2473,7230-001,Others,0.00,7.050755e+03,Detalle,NaN,-7050.755
2474,7230-002,Others,0.00,1.143691e+04,Detalle,NaN,-11436.910
2475,7230-003,Others,0.00,1.217892e+04,Detalle,NaN,-12178.920
2476,7230-004,Others,0.00,3.025870e+03,Detalle,NaN,-3025.870


In [51]:
#agrupamos todas las categorias que van dentro de shoot para ver el budget total

categorias_shoot = [
    'ALL SHOW', '2nd Unit', 'COVID', 'Fringes',
]

# Si Etapa es exactamente igual a alguna de estas, cambia a 'shoot'
final_cost_report["Etapa"] = final_cost_report["Etapa"].replace(categorias_shoot, "SHOOT")

final_cost_report["Etapa"] = np.where(
    final_cost_report["Etapa"].isin(categorias_shoot), 
    "SHOOT", 
    final_cost_report["Etapa"]
)



In [52]:
budget_por_etapa = final_cost_report.groupby("Etapa", as_index=False)["BUDGET"].sum()

# Ordenar de mayor a menor 
budget_por_etapa = budget_por_etapa.sort_values("BUDGET", ascending=False)

budget_por_etapa["BUDGET"] = budget_por_etapa["BUDGET"].apply(lambda x: f"${x:,.0f}")

print("\nPresupuesto asignado por etapa:")
print(budget_por_etapa.to_string(index=False))



Presupuesto asignado por etapa:
      Etapa       BUDGET
      SHOOT $100,328,918
       POST  $11,659,353
DEVELOPMENT  $10,171,000
       PREP   $9,104,498
       WRAP   $2,877,097
       SOFT     $957,045


Sacamos el budget por tipo y sacamos el porcentaje de ahorro en comparacion a su budget

In [53]:
# Agrupar por 'Etapa' y sumar Budget y Cost to Date
ahorro_etapa = final_cost_report.groupby("Etapa", as_index=False)[["BUDGET", "COST TO DATE"]].sum()

# Calcular la diferencia y el porcentaje de ahorro
ahorro_etapa["DIFERENCIA"] = ahorro_etapa["BUDGET"] - ahorro_etapa["COST TO DATE"]
ahorro_etapa["% AHORRO"] = ((ahorro_etapa["DIFERENCIA"]) / ahorro_etapa["BUDGET"]) * 100

# Formatear columnas para que luzcan bonitas
ahorro_etapa["BUDGET"] = ahorro_etapa["BUDGET"].apply(lambda x: f"${x:,.0f}")
ahorro_etapa["COST TO DATE"] = ahorro_etapa["COST TO DATE"].apply(lambda x: f"${x:,.0f}")
ahorro_etapa["DIFERENCIA"] = ahorro_etapa["DIFERENCIA"].apply(lambda x: f"${x:,.0f}")
ahorro_etapa["% AHORRO"] = ahorro_etapa["% AHORRO"].apply(lambda x: f"{x:.2f}%")

print("\nPorcentaje de ahorro por etapa:")
print(ahorro_etapa.to_string(index=False))




Porcentaje de ahorro por etapa:
      Etapa       BUDGET COST TO DATE DIFERENCIA % AHORRO
DEVELOPMENT  $10,171,000  $10,165,540     $5,460    0.05%
       POST  $11,659,353  $10,835,362   $823,991    7.07%
       PREP   $9,104,498   $9,163,160   $-58,662   -0.64%
      SHOOT $100,328,918 $100,565,711  $-236,793   -0.24%
       SOFT     $957,045     $983,295   $-26,250   -2.74%
       WRAP   $2,877,097   $2,597,238   $279,859    9.73%
